In [35]:
from langchain.chat_models import ChatOpenAI
from langchain.memory import ConversationBufferMemory
from langchain.storage import LocalFileStore
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings, CacheBackedEmbeddings
from langchain.vectorstores import FAISS
from langchain.prompts import PromptTemplate, ChatPromptTemplate, MessagesPlaceholder
from langchain.chains import RetrievalQA, ConversationalRetrievalChain
from langchain.document_loaders import UnstructuredFileLoader

llm = ChatOpenAI(temperature=0.1)

cached_dir = LocalFileStore("./.cache/")

In [3]:
splitter = CharacterTextSplitter.from_tiktoken_encoder(
    separator="\n", 
    chunk_size = 600, 
    chunk_overlap = 100
    )

loader = UnstructuredFileLoader("./files/chapter_3.pdf")

docs = loader.load_and_split(text_splitter=splitter)

In [9]:
embeddings = OpenAIEmbeddings()

cached_embeddings = CacheBackedEmbeddings.from_bytes_store(embeddings, cached_dir)

vectorstore = FAISS.from_documents(docs, cached_embeddings)

In [67]:
prompt_template = """
Use the following context and conversation history to answer the question.

Context:
{context}

Conversation History:
{chat_history}

Question:
{query}

Answer:
"""

prompt = PromptTemplate(
    input_variables=["context", "chat_history", "query"],
    template=prompt_template,
)

In [72]:
memory = ConversationBufferMemory(
    memory_key="chat_history",
    return_messages=True,
)

In [73]:
chain = ConversationalRetrievalChain.from_llm(
    llm=llm,
    retriever=vectorstore.as_retriever(),
    memory=memory,
    verbose=True
)

In [74]:
response = chain.run("Aaronson은 유죄인가요?")
print(response)



> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
System: Use the following pieces of context to answer the user's question. 
If you don't know the answer, just say that you don't know, don't try to make up an answer.
----------------
his arm round her waist.
There was no telescreen, but there must be hidden microphones: besides, they could be seen. It did not
matter, nothing mattered. They could have lain down on the ground and done THAT if they had wanted to.
His flesh froze with horror at the thought of it. She made no response whatever to the clasp of his arm; she
did not even try to disengage herself. He knew now what had changed in her. Her face was sallower, and
there was a long scar, partly hidden by the hair, across her forehead and temple; but that was not the
change. It was that her waist had grown thicker, and, in a surprising way, had stiffened. He remembered
16 / 20
chapter_3.md
2025-03-23
how once, after the explosi

In [75]:
response = chain.run("그가 테이블에 어떤 메시지를 썼나요?")
print(response)



> Entering new LLMChain chain...
Prompt after formatting:
Given the following conversation and a follow up question, rephrase the follow up question to be a standalone question, in its original language.

Chat History:

Human: Aaronson은 유죄인가요?
Assistant: 네, Jones, Aaronson, 그리고 Rutherford는 자신들에게 죄를 물렸다고 합니다.
Follow Up Input: 그가 테이블에 어떤 메시지를 썼나요?
Standalone question:

> Finished chain.


> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
System: Use the following pieces of context to answer the user's question. 
If you don't know the answer, just say that you don't know, don't try to make up an answer.
----------------
recognizable from behind.
17 / 20
chapter_3.md
2025-03-23
'At the time when it happens,' she had said, 'you do mean it.' He had meant it. He had not merely said it, he
had wished it. He had wished that she and not he should be delivered over to the----
Something changed in the music that trickled from the telescreen. 

In [77]:
chain.run("Julia 는 누구인가요?")



> Entering new LLMChain chain...
Prompt after formatting:
Given the following conversation and a follow up question, rephrase the follow up question to be a standalone question, in its original language.

Chat History:

Human: Aaronson은 유죄인가요?
Assistant: 네, Jones, Aaronson, 그리고 Rutherford는 자신들에게 죄를 물렸다고 합니다.
Human: 그가 테이블에 어떤 메시지를 썼나요?
Assistant: 그가 테이블에 쓴 메시지는 "FREEDOM IS SLAVERY"와 "TWO AND TWO MAKE FIVE" 그리고 "GOD IS POWER"입니다.
Follow Up Input: Julia 는 누구인가요?
Standalone question:

> Finished chain.


> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
System: Use the following pieces of context to answer the user's question. 
If you don't know the answer, just say that you don't know, don't try to make up an answer.
----------------
walk three kilometres, measured by pacing the cell, and his bowed shoulders were growing straighter. He
attempted more elaborate exercises, and was astonished and humiliated to find what things he could

'Julia는 Winston과 사랑을 나누었던 여성으로, 이야기에서 중요한 역할을 하는 캐릭터입니다.'

In [78]:
memory.load_memory_variables({})

{'chat_history': [HumanMessage(content='Aaronson은 유죄인가요?'),
  AIMessage(content='네, Jones, Aaronson, 그리고 Rutherford는 자신들에게 죄를 물렸다고 합니다.'),
  HumanMessage(content='그가 테이블에 어떤 메시지를 썼나요?'),
  AIMessage(content='그가 테이블에 쓴 메시지는 "FREEDOM IS SLAVERY"와 "TWO AND TWO MAKE FIVE" 그리고 "GOD IS POWER"입니다.'),
  HumanMessage(content='Julia 는 누구인가요?'),
  AIMessage(content='Julia는 Winston과 사랑을 나누었던 여성으로, 이야기에서 중요한 역할을 하는 캐릭터입니다.')]}